In [1]:
import numpy as np
import pandas as pd

import csv

from pymorphy2 import MorphAnalyzer

In [2]:
test = pd.read_csv('gapping/test.csv', sep = '\t', quoting = csv.QUOTE_NONE)

In [3]:
res = pd.read_csv('results/result.csv', sep = '\t', quoting = csv.QUOTE_NONE)

In [4]:
res['text'] = test['text']  # баг с сохранением в моей модели (сохранялось без quoting'а, поэтому добавлено это)

In [5]:
def find_left_part(sent, left_index):
    # принимает предложение и левый индекс; возвращает "левый предикат"
    rres = left_index
    bflag = 0
    for symbol in sent[:left_index][-1::-1]:
        if symbol.isalpha() or symbol.isdigit():
            bflag = 1
            break
        else:
            rres = rres - 1
    if bflag == 0:
        return '-1:-1'
    lres = rres
    last_digit = 1 if sent[rres].isdigit() else 0 # flag
    for symbol in sent[:rres][-1::-1]:
        if symbol in ['[', ']', '(', ')', ':', '^', "'", '"']:
            break
        elif (symbol in [',', '.']) and (last_digit == 0):
            break
        else:
            lres = lres - 1
            if symbol.isdigit():
                last_digit = 1
            else:
                last_digit = 0
    for symbol in sent[lres:rres]:
        if symbol.isalpha() or symbol.isdigit():
            break
        else:
            lres = lres + 1
    # далее удаление стартового союза, если такой имеется
    text = sent[lres:rres]
    cres = lres
    word = ''
    flag = 0
    for char in sent[lres:rres]:
        if char.isalpha():
            word = word + char
            cres = cres + 1
        else:
            flag = 1
            break
    if (flag == 1):
        morph = MorphAnalyzer()
        if morph.parse(word)[0].tag.POS == 'CONJ':
            lres = cres
            for symbol in sent[lres:rres]:
                if symbol.isalpha() or symbol.isdigit():
                    break
                else:
                    lres = lres + 1
    return f"{lres}:{rres}"

In [6]:
def find_right_part(sent, right_index):
    lres = right_index + 1
    bflag = 0
    for symbol in sent[lres:]:
        if symbol.isalpha() or symbol.isdigit():
            bflag = 1
            break
        else:
            lres = lres + 1
    if bflag == 0:
        return '-1:-1'
    rres = lres
    if lres == 0:
        last_dsep = 0
    else:
        last_dsep = 1 if (sent[lres-1] == ',' or sent[lres-1] == '.') else 0 # flag
    for symbol in sent[lres:]:
        if last_dsep == 1:
            if not symbol.isdigit():
                break
            else:
                rres = rres + 1
        if symbol in ['[', ']', '(', ')', ':', '^', "'", '"']:
            break
        elif symbol in [',', '.']:
            last_dsep = 1
        else:
            rres = rres + 1
            last_dsep = 0
    for symbol in sent[lres:rres][-1::-1]:
        if symbol.isalpha() or symbol.isdigit():
            break
        else:
            rres = rres - 1
    return f"{lres}:{rres}"

In [7]:
def find_r(row):
    if row['class'] == 0:
        row['cR1'] = '-1:-1'
        row['cR2'] = '-1:-1'
        row['R1'] = '-1:-1'
        row['R2'] = '-1:-1'
    else:
        res_r1 = ''
        res_r2 = ''
        for v_range in row['V'].strip().split(' '):
            res_r1 = res_r1 + find_left_part(row['text'], int(v_range.split(':')[0])) + ' '
            res_r2 = res_r2 + find_right_part(row['text'], int(v_range.split(':')[1]) - 1) + ' '
        row['cR1'] = find_left_part(row['text'], int(row['cV'].split(':')[0]))
        row['cR2'] = find_right_part(row['text'], int(row['cV'].split(':')[1]))
        row['R1'] = res_r1
        row['R2'] = res_r2
    return row

In [8]:
res = res.apply(find_r, axis = 1)

In [9]:
res[['text', 'class', 'cV', 'cR1', 'cR2', 'V', 'R1', 'R2']].to_csv('results/final_result.csv', sep = '\t', index = None, quoting=csv.QUOTE_NONE)